In [4]:
import sys


if '../src' in sys.path:
    sys.path.remove('../src')

sys.path.append('../src')

from board import create_board, make_move, check_win
from player import random_player, heuristic_player, intelligent_player
from simulation import simulate_game

players = {
    "4 Years": random_player,
    "7 Years": heuristic_player,
    "9 Years": lambda b,p: intelligent_player(b,p,depth=1),
    "11 Years": lambda b,p: intelligent_player(b,p,depth=2)
}

for p1_name, p2_name in combinations(players.keys(), 2):
    sims = 100
    outcomes = {p1_name:0, p2_name:0, "Draw":0}
    for _ in range(sims):
        winner = simulate_game(players[p1_name], players[p2_name])
        if winner==1: outcomes[p1_name]+=1
        elif winner==2: outcomes[p2_name]+=1
        else: outcomes["Draw"]+=1
    prob1 = outcomes[p1_name]/sims
    prob2 = outcomes[p2_name]/sims
    draw = outcomes["Draw"]/sims
    se1 = (prob1*(1-prob1)/sims)**0.5
    se2 = (prob2*(1-prob2)/sims)**0.5
    sed = (draw*(1-draw)/sims)**0.5
    print(f"\n{p1_name} vs {p2_name} ({sims} games)")
    print(f"{p1_name}: {prob1*100:.1f}% ± {se1*100:.1f}%")
    print(f"{p2_name}: {prob2*100:.1f}% ± {se2*100:.1f}%")
    print(f"Draw: {draw*100:.1f}% ± {sed*100:.1f}%")


['C:\\Program Files\\JetBrains\\PyCharm 2025.3.1\\plugins\\python-ce\\helpers\\jupyter_debug', 'C:\\Program Files\\JetBrains\\PyCharm 2025.3.1\\plugins\\python-ce\\helpers\\pydev', 'C:\\Users\\admin\\AppData\\Local\\Python\\pythoncore-3.14-64\\python314.zip', 'C:\\Users\\admin\\AppData\\Local\\Python\\pythoncore-3.14-64\\DLLs', 'C:\\Users\\admin\\AppData\\Local\\Python\\pythoncore-3.14-64\\Lib', 'C:\\Users\\admin\\AppData\\Local\\Python\\pythoncore-3.14-64', '', 'C:\\Users\\admin\\AppData\\Local\\Python\\pythoncore-3.14-64\\Lib\\site-packages', 'C:\\Users\\admin\\PycharmProjects\\connect_four_monte_carlo\\notebook\\src', 'C:\\Users\\admin\\PycharmProjects\\connect_four_monte_carlo\\notebook\\..\\src']


NameError: name 'combinations' is not defined